# Systems genetics 2020
## Final Project  

In [8]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [9]:
liver_exp = pd.read_csv('data/GSE17522_series_matrix_liver.txt', sep = "\t")
brain_exp = pd.read_csv('data/GSE36674_series_matrix_hypothalamus.txt', sep = "\t")
liver_annotations = pd.read_csv('data/annotations_liver_GPL6466-9752.txt', sep = "\t")
genotypes = pd.read_excel('data/genotypes.xls', headers=None)
pheotypes = pd.read_excel('data/phenotypes.xls')
mgi = pd.read_csv('data/MGI_Coordinates.Build37.rpt.txt', sep = "\t", error_bad_lines=False, warn_bad_lines=False)

### 2. Gene expression data preprocessing

#### • Make sure that the data is normalized

In [10]:
liver_exp[liver_exp['!Sample_title'] == '!Sample_data_processing']['Liver_C57BL6J_M_B1_rep1'].iloc[0]

'Data were extracted from the scanned image using Agilent Feature Extraction software version 6.1. A total of 122 arrays were run in 8 batches. The samples were semirandomly distributed throughout the batches prior to microarray analysis in order to separate sexes and strains, and to minimize between- and within- batch bias. Technical and biological replicates were run both within each batch and between batches. The microarray data was deposited in the UNC Microarray Database and extracted using Log2 ratios of the mean red channel intensity over the mean green channel intensity. This was followed by LOWESS normalization to remove the intensity dependent dye bias3. Neither the genes nor the arrays were centered. Inter-batch normalization was carried out using a nested ANOVA mixed model with samples within each batch crossed with sex and strain.'

In [11]:
brain_exp[brain_exp['!Sample_title'] == '!Sample_data_processing']['BXD44_F'].iloc[0]

'Probe intensity values were extracted using the Affymetrix GeneChip Operating Software and RMA normalized'

#### • Merge data file with annotation file to get your input matrix

In [12]:
# Remove metadata rows to get the raw data only
liver_exp = liver_exp[~liver_exp['!Sample_title'].str.contains('!Sample_', na=False)]
liver_exp = liver_exp[~liver_exp['!Sample_title'].str.contains('!series_', na=False)]
liver_exp = liver_exp[liver_exp['!Sample_title'] != 'ID_REF']

# Rename ID column to match the annotation matrix
liver_exp = liver_exp.rename(columns = {'!Sample_title' : 'ID'})

# Merge with annotation matrix to get the gene identifier (gene symbol)
id_cols = ['ID', 'GENE_SYMBOL']
liver_annotations = liver_annotations[id_cols]
liver_matrix = liver_annotations.merge(liver_exp, left_on='ID', right_on='ID')

# Keep BXD columns and identifier columns only
liver_bxd_cols = list(liver_matrix.filter(regex=("BXD*")).columns)
liver_matrix = liver_matrix[id_cols + liver_bxd_cols]

# Rename BXD columns
liver_matrix = liver_matrix.rename(columns={col : col.split('_')[1] for col in liver_matrix.columns.drop(id_cols)})

#### • Remove rows with no gene identifier, <br> • Remove rows with low maximal value.  <br> • Remove rows with low variance.


In [13]:
# Remove rows with no gene identifier
liver_matrix = liver_matrix[~liver_matrix['GENE_SYMBOL'].isna()]
liver_matrix

,ID,GENE_SYMBOL,BXD1,BXD12,BXD23,BXD34,BXD28,BXD29,BXD39,BXD40,...,BXD39,BXD86,BXD2,BXD69,BXD73,BXD92,BXD51,BXD48,BXD85,BXD45
0,A_51_P100021,Hivep3,-0.011,-0.075,-0.091,-0.148,0.052,0.06,-0.09,-0.047,...,0.061,-0.21,0.239,-0.06,0.305,0.139,0.053,0.099,0.044,0.137
1,A_51_P100034,Mif4gd,-0.641,-0.396,0.399,-0.892,-0.326,0.002,-0.406,-0.642,...,0.146,-0.216,-0.419,-0.275,-0.441,-0.58,-0.187,-0.234,-0.538,-0.545
2,A_51_P100052,Slitrk2,0.192,0.367,0.789,0.583,0.584,0.663,0.515,0.391,...,0.414,0.234,0.43,0.233,0.612,0.087,0.026,0.393,0.208,0.247
3,A_51_P100065,Lnx1,-1.589,-1.777,-1.648,-1.68,-1.834,-1.608,-1.816,-1.021,...,-2.023,-2.182,-2.522,-2.401,-1.741,-1.91,-2.115,-1.877,-2.316,-2.343
5,A_51_P100099,2610002J23Rik,-1.19,-1.013,-1.142,-1.312,-1.163,-0.983,-1.31,-1.042,...,-1.158,-1.209,-1.208,-0.784,-0.869,-0.848,-1.11,-1.159,-0.857,-1.036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20863,A_51_P521307,Olfr1512,0.027,0.245,0.28,0.334,0.119,0.456,0.595,0.473,...,0.185,0.1,0.231,0.359,0.286,0.24,0.227,0.292,0.026,-0.145
20864,A_51_P521346,C7,1.101,0.931,0.724,1.054,0.896,0.876,1.064,0.986,...,0.841,0.758,0.831,1.14,0.618,0.679,0.823,0.678,0.88,0.874
20865,A_51_P521364,Emx2,0.129,0.353,0.373,0.496,0.328,0.204,0.475,0.59,...,0.306,0.249,0.347,0.517,0.207,0.322,0.33,0.246,0.301,0.23
20866,A_51_P521374,Nudt13,-0.426,-0.363,-0.511,-0.423,-0.414,0.033,-0.047,-0.357,...,-0.108,0.028,-0.146,-0.216,-0.273,-0.409,-0.327,-0.215,-0.505,-0.261


### Brain.....

In [7]:
# Remove metadata rows to get the raw data only
brain_exp = brain_exp[~brain_exp['!Sample_title'].str.contains('!Sample_', na=False)]
brain_exp = brain_exp[~brain_exp['!Sample_title'].str.contains('!series_', na=False)]
brain_exp = brain_exp[brain_exp['!Sample_title'] != 'ID_REF']

# Rename ID column to match the annotation matrix
brain_exp = brain_exp.rename(columns = {'!Sample_title' : 'ID'})
brain_exp

,ID,BXD44_F,BXD49_M,BXD44_M,BXD102_M,BXD43_M,BXD45_F,BXD45_M,BXD103_M,BXD103_F,...,BXD92A_F,BXD70_F,BXD92A_M,BXD68_F,BXD68_M,BXD79_F,BXD85_F,BXD85_M,BXD101_M,BXD101_F
34,10338001,12.612,12.509,12.629,12.56,12.533,12.646,12.426,12.465,12.728,...,12.451,12.812,12.757,12.697,12.943,12.622,12.963,12.522,12.558,12.872
35,10338003,11.187,11.043,11.116,11.104,11.09,11.151,10.955,11.06,11.256,...,10.975,11.304,11.207,11.246,11.445,11.15,11.539,10.98,11.093,11.314
36,10338004,10.021,9.627,9.875,9.817,10.031,9.906,9.766,9.881,10.064,...,9.83,9.929,9.972,9.929,10.065,9.772,9.973,9.926,9.799,10.021
37,10338017,13.598,13.342,13.509,13.518,13.495,13.516,13.4,13.42,13.594,...,13.562,13.723,13.667,13.543,13.811,13.482,13.651,13.559,13.575,13.827
38,10338025,9.937,9.654,9.743,9.868,9.854,9.866,9.719,9.799,9.948,...,9.866,10,9.892,10.007,10.092,9.809,10.074,9.828,9.967,10.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34789,10608720,9,9.033,9.154,8.93,9.274,8.834,9.065,9.121,9.103,...,9.085,9.136,9.199,9.265,9.126,9.092,9.342,8.994,9.025,9.337
34790,10608721,9.121,9.608,9.108,9.218,8.926,9.228,9.177,9.078,9.042,...,9.09,9.185,8.891,9.319,9.483,9.093,8.923,9.135,9.028,8.967
34791,10608722,5.691,5.466,5.591,5.674,5.831,5.541,5.618,5.604,5.553,...,5.522,5.379,5.525,5.573,5.323,5.61,5.621,5.657,5.591,5.336
34792,10608723,5.368,5.335,5.23,5.245,5.373,5.199,5.255,5.282,5.161,...,5.109,5.229,5.157,5.057,5.181,5.139,5.157,5.079,5.142,4.94
